In [2]:
import numpy as np

In [3]:
num_states = 3
num_actions_per_player = 2
num_actions = num_actions_per_player**2
num_trans = 3
reward_range = [-1,1]

def generate_random_trans_and_rewards():
    trans_prob_matrices = []
    reward_matrices = []
    for _ in range(num_trans):
        trans_prob_matrix = []
        reward_matrix = []
        for s in range(num_states):
            trans_prob_matrix_for_s = []
            reward_matrix_for_s = []
            for a in range(num_actions):
                rands = np.random.uniform(0,1, num_states)
                rand_probs = list(rands/sum(rands))
                trans_prob_matrix_for_s.append(rand_probs)
                rs = np.random.uniform(*reward_range, num_states)
                reward_matrix_for_s.append(list(rs))
            trans_prob_matrix.append(trans_prob_matrix_for_s)
            reward_matrix.append(reward_matrix_for_s)
        trans_prob_matrices.append(trans_prob_matrix)
        reward_matrices.append(reward_matrix)
    return trans_prob_matrices, reward_matrices

tpmxs, rmxs =  generate_random_trans_and_rewards()  # shape: (trans, state, action, next_state)
print(np.array(tpmxs).shape, np.array(rmxs).shape, rmxs) # shape: (trans, state, action, next_state)


(3, 3, 4, 3) (3, 3, 4, 3) [[[[0.21123743774355774, 0.1711753238597562, -0.17610521369565313], [-0.4467982890426656, 0.06548356678565703, -0.1651711687075217], [-0.037786825907280885, -0.6662661898215996, 0.41012368726779735], [-0.47343150573854764, 0.86270836668386, 0.9486577603738413]], [[-0.2876291306053398, -0.504835375531637, 0.09335899326261665], [-0.11356981306151481, 0.3960231516678929, 0.06363797880791311], [-0.7770320290713522, -0.27875838330354674, -0.4865296776499146], [0.294644435117732, -0.38371822115834275, 0.11812439744915082]], [[-0.3808343319214458, 0.18090467396156962, -0.5704275828534058], [0.782374497590993, 0.5581036449940866, 0.8048557868956976], [-0.8478519511468259, 0.6469293284244844, -0.45941974101210725], [-0.01790355624675577, -0.7286287645361322, 0.8207678544839876]]], [[[0.4245523326939702, 0.13727141303554635, -0.7295434104126808], [-0.59054614290371, 0.0769522669560021, 0.3228690622512722], [0.4325722039205977, 0.5922347700709569, 0.6382580108128053], [0

In [4]:
import ecos
from scipy.sparse import csr_matrix

def NashEquilibriumECOSSolver(M):
    """
    https://github.com/embotech/ecos-python
    min  c*x
    s.t. A*x = b
         G*x <= h
    https://github.com/embotech/ecos/wiki/Usage-from-MATLAB
    args: 
        c,b,h: numpy.array
        A, G: Scipy sparse matrix
    """
    row, col = M.shape
    c = np.zeros(row+1)
    # max z
    c[-1] = -1  
    
    # x1+x2+...+xn=1
    A = np.ones(row+1)
    A[-1] = 0.
    A = csr_matrix([A])
    b=np.array([1.])
    
    # M.T*x<=z
    G1 = np.ones((col, row+1))
    G1[:col, :row] = -1. * M.T
    # x>=0
    G2 = np.zeros((row, row+1))
    for i in range(row):
        G2[i, i]=-1. 
    # x<=1.
    G3 = np.zeros((row, row+1))
    for i in range(row):
        G3[i, i]=1. 
    G = csr_matrix(np.concatenate((G1, G2, G3)))
    h = np.concatenate((np.zeros(2*row), np.ones(row)))
    
    # specify number of variables
    dims={'l': col+2*row, 'q': []}
                       
    solution = ecos.solve(c,G,h,dims,A,b, verbose=False)

    p1_value = solution['x'][:row]
    p2_value = solution['z'][:col] # z is the dual variable of x
    # There are at least two bad cases with above constrained optimization,
    # where the constraints are not fully satisfied (some numerical issue):
    # 1. the sum of vars is larger than 1.
    # 2. the value of var may be negative.
    abs_p1_value = np.abs(p1_value)
    abs_p2_value = np.abs(p2_value)
    p1_value = abs_p1_value/np.sum(abs_p1_value)
    p2_value = abs_p2_value/np.sum(abs_p2_value)

    return (p1_value, p2_value)

In [13]:
class ArbitraryMDP():
    def __init__(self, num_states=3, num_actions_per_player=2, num_trans=3):
        self.num_states = num_states
        self.num_actions_per_player = num_actions_per_player
        self.num_actions = self.num_actions_per_player**2
        self.num_trans = num_trans
        self.reward_range = [-1,1]
        self.state = None
        self._construct_game()

    def _construct_game(self, ):
        self.trans_prob_matrices, self.reward_matrices = self.generate_random_trans_and_rewards()

    def generate_random_trans_and_rewards(self,):
        trans_prob_matrices = []
        reward_matrices = []
        for _ in range(self.num_trans):
            trans_prob_matrix = []
            reward_matrix = []
            for s in range(self.num_states):
                trans_prob_matrix_for_s = []
                reward_matrix_for_s = []
                for a in range(self.num_actions):
                    rands = np.random.uniform(0,1, self.num_states)
                    rand_probs = list(rands/sum(rands))
                    trans_prob_matrix_for_s.append(rand_probs)
                    rs = np.random.uniform(*self.reward_range, self.num_states)
                    reward_matrix_for_s.append(list(rs))
                trans_prob_matrix.append(trans_prob_matrix_for_s)
                reward_matrix.append(reward_matrix_for_s)
            trans_prob_matrices.append(trans_prob_matrix)
            reward_matrices.append(reward_matrix)

        return trans_prob_matrices, reward_matrices

    def reset(self, ):
        self.state = np.random.randint(0, self.num_states)  # randomly pick one state as initial
        self.trans = 0
        obs = self.state
        return obs

    def step(self, a):
        """The environment transition function.
        For a given state and action, the transition is stochastic. 
        For representation of states, considering the num_states=3 case, the first three states are (0,1,2);
        after one transition, the possible states are (3,4,5), etc. Such that states after different numbers of 
        transitions can be distinguished. 

        :param a: action
        """
        trans_prob = self.trans_prob_matrices[self.trans][self.state%self.num_states][a]
        next_state = np.random.choice([i for i in range(self.num_states)], p=trans_prob) + (self.trans+1) * self.num_states
        reward = self.reward_matrices[self.trans][self.state%self.num_states][a][next_state%self.num_states]

        self.state = next_state
        obs = self.state
        self.trans += 1
        done = False if self.trans < self.num_trans else True

        return obs, reward, done, None

    def NEsolver(self,):
        self.Nash_v = []
        self.Nash_strategies = []
        for tm, rm in zip(self.trans_prob_matrices[::-1], self.reward_matrices[::-1]): # inverse enumerate 
            if len(self.Nash_v) > 0:
                rm = np.array(rm)+np.array(self.Nash_v[-1])  # broadcast sum on rm's last dim, last one in Nash_v is for the next state
            trm = np.einsum("ijk,ijk->ij", tm, rm)  # transition prob * reward for the last dimension in (state, action, next_state)
            trm = trm.reshape(-1, self.num_actions_per_player, self.num_actions_per_player) # action list to matrix
            ne_values = []
            ne_strategies = []
            for s_payoff in trm:
                ne = NashEquilibriumECOSSolver(s_payoff)
                ne_strategies.append(ne)
                ne_value = ne[0]@s_payoff@ne[1].T
                ne_values.append(ne_value)  # each value is a Nash equilibrium value on one state
            self.Nash_v.append(ne_values)  # (trans, state)
            self.Nash_strategies.append(ne_strategies)
        self.Nash_v = self.Nash_v[::-1]
        print('Nash strategies of all states: ', self.Nash_strategies)
        self.Nash_strategies = self.Nash_strategies[::-1]
        print('Nash values of all states: ', self.Nash_v)
        print('Nash strategies of all states: ', self.Nash_strategies)

env = ArbitraryMDP()
env.NEsolver()
obs = env.reset()
print('ini: ', obs)    

for _ in range(env.num_trans+1):
    o,r,d,_ = env.step(1)
    print(o,r,d)
    if d:
        break


Nash values of all states:  [[-0.07034894510037229, -0.12165961496544798, -0.1231529747442372], [-0.11650771089793736, -0.4427388156496247, -0.2291649596228951], [0.047601410235858975, -0.5273669918931514, 0.5957175655384962]]
Nash strategies of all states:  [[(array([0.87662394, 0.12337606]), array([0.94613562, 0.05386438])), (array([1.84944776e-11, 1.00000000e+00]), array([4.85412952e-11, 1.00000000e+00])), (array([0.33493167, 0.66506833]), array([0.45890273, 0.54109727]))], [(array([3.1754892e-11, 1.0000000e+00]), array([1.23189702e-11, 1.00000000e+00])), (array([0.77744955, 0.22255045]), array([0.48238818, 0.51761182])), (array([6.83524686e-10, 9.99999999e-01]), array([1.00000000e+00, 2.04625125e-10]))], [(array([1.94368420e-09, 9.99999998e-01]), array([9.99999999e-01, 5.24423023e-10])), (array([1.38224865e-10, 1.00000000e+00]), array([9.48760378e-11, 1.00000000e+00])), (array([8.49308535e-12, 1.00000000e+00]), array([4.27167993e-11, 1.00000000e+00]))]]
ini:  1
4 -0.811331066258497

/home/quantumiracle/anaconda3/envs/res/lib/python3.6/site-packages/ecos/ecos.py:26: UserWarning: Converting G to a CSC matrix; may take a while.
  warn("Converting G to a CSC matrix; may take a while.")
/home/quantumiracle/anaconda3/envs/res/lib/python3.6/site-packages/ecos/ecos.py:29: UserWarning: Converting A to a CSC matrix; may take a while.
  warn("Converting A to a CSC matrix; may take a while.")


In [16]:
def kl(p, q):
    """Kullback-Leibler divergence D(P || Q) for discrete distributions
    Parameters
    ----------
    p, q : array-like, dtype=float, shape=n
    Discrete probability distributions.
    """
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)

    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

a=np.array([0.5,0.5])
b=np.array([0.,1.])
kl(a,b)

/home/quantumiracle/anaconda3/envs/res/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


inf

In [22]:
np.arange(2)

array([0, 1])